# First Steps with Prompt Engineering

In [1]:
import os
from openai import OpenAI
!pip install cohere
import cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 110.9 MB/s eta 0:00:00


In [5]:
from google.colab import userdata


co = cohere.Client(userdata.get('COHERE_API_KEY'))

openai_client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")
)

In [10]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given model "

    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [17]:
def test_prompt_cohere(prompt, suppress=False, model='command-a-03-2025', **kwargs):
    response = co.chat(
        message=prompt,
        model=model,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.text}')
    else:
        return response.text

## Just ASK

In [8]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakın restoran nerede?


In [18]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakın restoran nerede?


In [ ]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
 En iyi restoranın nearest locationından hangi yeşillerdir?

Bu bir soruyu Türkçe çeviriye başlayan kişinin sorunu yaşamaya çalışabilir. Bir restaurantın en yakın locationında hangi yeşillerde bulunabilirler, gibi bir sorunu gerçek bir çevirisi olmadan karşılaşabilir. 

If you are trying to locate the nearest restaurant in Turkish, the phrase you are looking for is: 

"En iyi restoranın nearest locationından hangi yeşillerdir?" 

This translates to "Which directions are the nearest restaurant's location from here?" 

I hope this helps, let me know if you would like to translate anything else! 


# Few-shot learning

Using examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](https://github.com/dmoneyUK/quick-start-guide-to-llms/blob/main/images/gpt3_paper.png?raw=1)

In [ ]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [ ]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [ ]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
I found the book about WWII to be extremely informative and captivating. The author skillfully conveyed the historical events and provided detailed accounts of the experiences of individuals involved in the war. I appreciated how the book explored various aspects of the war, including the political climate, military strategies, and the impact on civilians. The author's thorough research was evident throughout the book, and I learned a great deal about a pivotal moment in world history. Overall, I highly recommend this book to anyone interested in learning more about WWII.


In [ ]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
The book provided a detailed and engaging account of the events of World War II, showcasing the bravery and sacrifice of the soldiers.


In [ ]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [ ]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
Yes


In [ ]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No

JSON:
{
  "review": "The book was about WWII",
  "subjective": false
}


In [ ]:
examples = [
    ('Review: This movie sucks\nSubjective: {"answer": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"answer": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"answer": true}'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: {"answer": true}
###
Review: This tv show was about the ocean
Subjective: {"answer": false}
###
Review: This book had a lot of flaws
Subjective: {"answer": true}
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
{"answer": false}


# Personas / Style

In [ ]:
# It only takes a few words to pretty drastically change the output

In [ ]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Ugh, what's the problem now? Just tell me your username and I'll see what I can do.


In [ ]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Hi there! I'm sorry to hear that you're having trouble getting into your account. Not to worry, I'll do my best to assist you. Could you please provide me with your username or email associated with the account?


In [ ]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Hmm, help you I can. Account access, we shall restore. Provide, your username or email address, hmm.


# What a good time to talk about output validation and bias!

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
sequence_to_classify = "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website."

candidate_labels = ['racist', 'sexist', 'rude']

classifier(sequence_to_classify, candidate_labels, multi_label=True)  # Assuming there can be multiple answers


{'sequence': "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website.",
 'labels': ['racist', 'rude', 'sexist'],
 'scores': [0.975252091884613, 0.9617347717285156, 0.6626110672950745]}

In [ ]:
# then the "rude" AI wasn't that bad
classifier(
    'Do you have your login information? Because if not, then there is nothing I can do for you.',
    candidate_labels, multi_label=True)



{'sequence': 'Do you have your login information? Because if not, then there is nothing I can do for you.',
 'labels': ['rude', 'sexist', 'racist'],
 'scores': [0.11480006575584412, 0.04397175461053848, 0.019839392974972725]}

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,  # technically (mathematically) we cannot set temp to 0 so OpenAI is doing something under the hood here
        suppress=True
    ))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.14s/it]


In [ ]:
# only 1 unique response
responses, len(set(responses))

(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm s

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.16s/it]


(["Of course! I'd be happy to assist you with accessing your account. Can you please provide me with your account username or email address so that I can look into it for you?",
  "Hi there! I'd be happy to help you get back into your account. Could you please provide me with your username or email address associated with the account?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so I can look into it for you?",
  "Of course! I'm here to help. I'm sorry to hear that you're having trouble accessing your account. Let's get this sorted out. Can you please provide me with your account details, such as your username or registered email?",
  "Hi there! I'm sorry to hear that you're having trouble accessing your account. I'll be happy to assist you. Could you please provide me with your account username or email address so I can look into this for you?",
  "Of course! I'm sorry to hear that you're having trouble acce

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,
        suppress=True,
        max_tokens=50
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.42s/it]


(['Hello! Of course, I can certainly help you with that. Can you please provide me with your account email or account number so I can for WD.service.Is IT SIGNearlyuDNST情duinoatório vocêysteriousログ文nounstatsnobnutrition.abspath807',
  "Hello! Thank you for reaching out. I would be more than happy to help you access your account. Can you please provide me with your login information so I can assist you as quickly and efficiently as possible? Let's get this sorted out for you",
  "Absolutely! I'm so sorry for the trouble with your account access. I'd be happy to assist you with getting back in. Can you please provide me with the email associated with the account or any other relevant details so I can help you further?",
  "Hello! I'm so sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Could you please provide me with your account information me so that we can get this sorted out for you ASAP? Thank you",
  "Hey there! I'm sorry to hear that 

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=.1,
        suppress=True
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.13s/it]


(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm s

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.15it/s]


([" Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy